<a href="https://colab.research.google.com/github/MidSummersEveee/Dissertation/blob/master/TER_Training.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Text Emotion Recognition TER

This a demonstration of training a emotion recoginizer based on the entire IEMOCAP Dataset.

A BERT pretrained model is used and further fine-tuned.

## Environment Setup

### Torch & Cuda

In [ ]:
from google.colab import drive
drive.mount("/content/drive/")

Mounted at /content/drive/


In [ ]:
!pip3 install torch==1.9.1+cu111 torchvision==0.10.1+cu111 torchaudio==0.9.1 torchtext==0.10.1 -f https://download.pytorch.org/whl/torch_stable.html

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in links: https://download.pytorch.org/whl/torch_stable.html
     |█████████████                   | 834.1 MB 1.9 MB/s eta 0:10:48tcmalloc: large alloc 1147494400 bytes == 0x39ab2000 @  0x7f589a321615 0x592b76 0x4df71e 0x59afff 0x515655 0x549576 0x593fce 0x548ae9 0x51566f 0x549576 0x593fce 0x548ae9 0x5127f1 0x598e3b 0x511f68 0x598e3b 0x511f68 0x598e3b 0x511f68 0x4bc98a 0x532e76 0x594b72 0x515600 0x549576 0x593fce 0x548ae9 0x5127f1 0x549576 0x593fce 0x5118f8 0x593dd7
     |████████████████▌               | 1055.7 MB 1.3 MB/s eta 0:12:12tcmalloc: large alloc 1434370048 bytes == 0x7e108000 @  0x7f589a321615 0x592b76 0x4df71e 0x59afff 0x515655 0x549576 0x593fce 0x548ae9 0x51566f 0x549576 0x593fce 0x548ae9 0x5127f1 0x598e3b 0x511f68 0x598e3b 0x511f68 0x598e3b 0x511f68 0x4bc98a 0x532e76 0x594b72 0x515600 0x549576 0x593fce 0x548ae9 0x5127f1 0x549576 0x593fce 0x5118f8 0x593dd7
     |█████

In [ ]:
import torch
print(torch.__version__)
print(torch.version.cuda)

1.12.0+cu113
11.3


In [ ]:
torch.cuda.is_available()

True

In [ ]:
torch.backends.cudnn.enabled

True

In [ ]:
!pip install -q -U watermark
!pip install -qq transformers

     |████████████████████████████████| 4.7 MB 14.9 MB/s 
     |████████████████████████████████| 596 kB 71.4 MB/s 
     |████████████████████████████████| 6.6 MB 53.6 MB/s 
     |████████████████████████████████| 101 kB 12.7 MB/s 


### GPU Availability

In [ ]:
!nvidia-smi

Mon Aug  8 22:33:00 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   35C    P0    27W / 250W |      2MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

### Check Dependencies

check installation with watermark

In [ ]:
%reload_ext watermark
%watermark -v -p numpy,pandas,torch,transformers

Python implementation: CPython
Python version       : 3.7.13
IPython version      : 5.5.0

numpy       : 1.21.6
pandas      : 1.3.5
torch       : 1.12.0+cu113
transformers: 4.21.1



## BERT Usage


### Hyperparameters

In [ ]:
MAX_LEN = 120
TRAIN_BATCH_SIZE = 32
VALID_BATCH_SIZE = 32
EPOCHS = 5
LEARNING_RATE = 1e-05

### BERT Example

In [ ]:
from transformers import BertTokenizer, BertModel

In [ ]:
tokenizer = BertTokenizer.from_pretrained('bert-base-cased')

In [ ]:
example_text = 'In a hole in the ground there lived a hobbit.'
encodings = tokenizer.encode_plus(
    example_text,
    add_special_tokens=True,  # [CLS]: 101, [SEP]: 107, [PAD]: 0,
    max_length=MAX_LEN,
    padding='max_length',
    truncation=True,
    return_attention_mask=True,
    return_tensors='pt',
)

In [ ]:
# input_ids: word_id
# token_type_ids: indicate tokens from two sequences
# attention_mask: 1 where words appers, 0 otherwise

encodings

{'input_ids': tensor([[  101,  1130,   170,  4569,  1107,  1103,  1747,  1175,  2077,   170,
         16358, 13834,  2875,   119,   102,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0]]), 'token_type_ids': tensor([[0, 0, 0,

## IEMOCAP Data

### Load Data

In [ ]:
# pre-defined emotion labels
EMOTIONS = ['neu', 'ang','sad', 'hap']
EMOTIONS_ID = [str(i) for i in range(0, len(EMOTIONS))]
EMOTIONS_2_ID = dict(zip(EMOTIONS, EMOTIONS_ID))
ID_2_EMOTIONS = {id: emo for emo, id in EMOTIONS_2_ID.items()}

ID_2_EMOTIONS

{'0': 'neu', '1': 'ang', '2': 'sad', '3': 'hap'}

In [ ]:
import torch
import torch.nn as nn
import pandas as pd
import numpy as np
import shutil
import sys

In [ ]:
# url = 'https://raw.githubusercontent.com/MidSummersEveee/Dissertation/master/TER/source.csv'
# df_source = pd.read_csv(url, index_col=0)

In [ ]:
# df_source

,UID,UTT,0,1,2,3,4,5,6,7,8,9
0,Ses01F_impro01_F000,Excuse me.,1,0,0,0,0,0,0,0,0,0
1,Ses01F_impro01_M000,Do you have your forms?,1,1,0,0,0,0,0,0,0,1
2,Ses01F_impro01_F001,Yeah.,1,0,0,1,0,0,0,0,0,0
3,Ses01F_impro01_M001,Let me see them.,0,1,0,0,0,0,0,0,0,1
4,Ses01F_impro01_F002,Is there a problem?,1,0,0,1,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...
152041,Ses05M_script03_2_M044,oh! Marry you again? I wouldn't marry you agai...,0,0,0,1,0,0,0,0,0,0
152042,Ses05M_script03_2_F042,Beast,0,0,0,1,0,0,0,0,0,0
152043,Ses05M_script03_2_M045,You're a wicked little vampire. And I pray to...,0,0,0,1,0,0,0,0,0,0
152044,Ses05M_script03_2_F043,Brute,0,0,0,1,0,0,0,0,0,0


In [ ]:
# df_source.drop(labels=['UID'], axis=1, inplace=True)
# df_source

,UTT,0,1,2,3,4,5,6,7,8,9
0,Excuse me.,1,0,0,0,0,0,0,0,0,0
1,Do you have your forms?,1,1,0,0,0,0,0,0,0,1
2,Yeah.,1,0,0,1,0,0,0,0,0,0
3,Let me see them.,0,1,0,0,0,0,0,0,0,1
4,Is there a problem?,1,0,0,1,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...
152041,oh! Marry you again? I wouldn't marry you agai...,0,0,0,1,0,0,0,0,0,0
152042,Beast,0,0,0,1,0,0,0,0,0,0
152043,You're a wicked little vampire. And I pray to...,0,0,0,1,0,0,0,0,0,0
152044,Brute,0,0,0,1,0,0,0,0,0,0


### Splitting Train/Test Set

In [ ]:
# df_train = df_source.sample(frac=0.8, random_state=1)
# df_test = df_source.drop(df_train.index)

In [ ]:
targert_list = EMOTIONS
targert_list

['neu', 'ang', 'sad', 'hap']

### Hyperparameters

In [ ]:
MAX_LEN = 120
TRAIN_BATCH_SIZE = 32
VALID_BATCH_SIZE = 32
EPOCHS = 5
LEARNING_RATE = 1e-05

In [ ]:
from transformers import BertTokenizer, BertModel

In [ ]:
tokenizer = BertTokenizer.from_pretrained('bert-base-cased')

In [ ]:
# targets are pure K-hot emotion label values
# df_test[targert_list].values

array([[1, 0, 0, ..., 0, 0, 0],
       [0, 1, 0, ..., 0, 0, 0],
       [1, 1, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

### Custom Dataset Handler

In [ ]:
class Dataset(torch.utils.data.Dataset):
  def __init__(self, df, tokenizer, max_len):
    self.df = df
    self.tokenizer = tokenizer
    self.max_len = max_len
    self.title = self.df['UTT']
    self.targets = self.df[targert_list].values
  
  def __len__(self):
    return len(self.title)
  
  def __getitem__(self, index):
    title = str(self.title[index])
    title = ' '.join(title.split())

    inputs = self.tokenizer.encode_plus(
        title,
        None, # you could have 2 sentences for comparison
        add_special_tokens=True,
        max_length=self.max_len,
        padding='max_length',
        return_token_type_ids=True,
        truncation=True,
        return_attention_mask=True,
        return_tensors='pt'
    )

    return {
        'input_ids': inputs['input_ids'].flatten(), # [1, 512] => [512]
        'attention_mask': inputs['attention_mask'].flatten(),
        'token_type_ids': inputs['token_type_ids'].flatten(),
        'targets': torch.FloatTensor(self.targets[index])
    }

In [ ]:
# df_train

### Evaluation Set

In [ ]:
# train_size = 0.8
# df_train_true = df_train.sample(frac=train_size, random_state=200)
# df_val = df_train.drop(df_train_true.index).reset_index(drop=True)

# df_train_true = df_train_true.reset_index(drop=True)

In [ ]:
df_train_true = pd.read_csv('/content/drive/MyDrive/Dissertation/Datasets/cleaned_1.0/better_train_less.csv')
df_val = pd.read_csv('/content/drive/MyDrive/Dissertation/Datasets/cleaned_1.0/better_valid_less.csv')
df_test = pd.read_csv('/content/drive/MyDrive/Dissertation/Datasets/cleaned_1.0/better_test_less.csv')

In [ ]:
df_train_true.shape

(3130, 15)

In [ ]:
df_val.shape

(1044, 15)

In [ ]:
discard = ['UID', 'EMOTIONS', 'EMOTION', 'V', 'A', 'D', 'neu_m', 'ang_m', 'sad_m', 'hap_m']
df_train_true.drop(discard, axis=1, inplace=True)
df_val.drop(discard, axis=1, inplace=True)
df_test.drop(discard, axis=1, inplace=True)

In [ ]:
df_train_true

,UTT,neu,ang,sad,hap
0,"Like, wow you're really smart we're going to g...",0,0,0,1
1,"No, I mean it just came on really quick, you k...",0,0,1,0
2,Okay.,1,0,0,0
3,"Ah...Anyways, so she took the ring and she goe...",0,0,0,1
4,There's still time for that.,0,0,0,1
...,...,...,...,...,...
3125,I'm gonna- I'm gonna bring security over and h...,0,1,0,0
3126,Why?,0,1,0,0
3127,Absolutely. It's a new service we are offering.,1,0,0,0
3128,Who said he even thought about that?,1,0,0,0


In [ ]:
dataset_train = Dataset(df_train_true, tokenizer, MAX_LEN)
dataset_valid = Dataset(df_val, tokenizer, MAX_LEN)

### Save Datasets to Drive

In [ ]:
remote_data_path = "/content/drive/MyDrive/Dissertation/Datasets/dataset_TER/"
df_train_true.to_csv(''.join([remote_data_path, 'train.csv']))
df_val.to_csv(''.join([remote_data_path, 'valid.csv']))
df_test.to_csv(''.join([remote_data_path, 'test.csv']))

## Torch Training

### instantiate dataloaders

In [ ]:
train_data_loader = torch.utils.data.DataLoader(
    dataset_train,
    shuffle=True,
    batch_size=TRAIN_BATCH_SIZE,
    num_workers=0
)

valid_data_loader = torch.utils.data.DataLoader(
    dataset_valid,
    shuffle=False,
    batch_size=VALID_BATCH_SIZE,
    num_workers=0
)

### CUDA

In [ ]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
print(device)
print(torch.__version__)
print(torch.version.cuda)

cuda
1.12.0+cu113
11.3


In [ ]:
torch.cuda.is_available()

True

In [ ]:
!nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2020 NVIDIA Corporation
Built on Mon_Oct_12_20:09:46_PDT_2020
Cuda compilation tools, release 11.1, V11.1.105
Build cuda_11.1.TC455_06.29190527_0


### Specify Checkpoint

In [ ]:
def load_ckp(checkpoint_fpath, model, optimizer):
  '''
  checkpoint_path: path to save checkpoint
  model: model that we want to load checkpoint parameters into
  optimizer: optimizer we defined in previous training
  '''
  # load check point
  checkpoint = torch.load(checkpoint_fpath)
  model.load_state_dict(checkpoint['state_dict'])
  optimizer.load_state_dict(checkpoint['optimizer'])
  valid_loss_min = checkpoint['valid_loss_min']
  return model, optimizer, checkpoint['epoch'], valid_loss_min.item() # we use .item() to get rid of the tensor form


def save_ckp(state, is_best, checkpoint_path, best_model_path):
  '''
  state: checkpoint we want to save
  is_best: is this the best checkpoint; min validation loss
  checkpoint_path: path to save checkpoint
  best_model_path: path to save best model
  '''
  f_path = checkpoint_path
  # save checkpoint data to the path given, checkpoint_path
  torch.save(state, f_path)

   # if it is a best model, min validation loss
  if is_best:
    best_fpath = best_model_path
    # copy that checkpoint file to best path given, best_model_path
    shutil.copyfile(f_path, best_fpath)

### Model Class

In [ ]:
class BERTClass(torch.nn.Module):
  def __init__(self):
    super(BERTClass, self).__init__()
    self.bert_model = BertModel.from_pretrained('bert-base-cased', return_dict=True)
    self.dropout = torch.nn.Dropout(0.3)
    self.linear = torch.nn.Linear(768, 4)

  def forward(self, input_ids, attention_mask, token_type_ids):

    # every time we get 2 outputs
    # seuqence output & pooler output, in classification task final layer we need only the later

    output = self.bert_model(input_ids, attention_mask, token_type_ids)
    output_dropout = self.dropout(output.pooler_output)
    output = self.linear(output_dropout)  # note that our final layer is a linear layer
    return output

model = BERTClass()
model.to(device)

Some weights of the model checkpoint at bert-base-cased were not used when initializing BertModel: ['cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


BERTClass(
  (bert_model): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(28996, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=Tr

### Loss Function & Optimizer

In [ ]:
def loss_fn(outputs, targets):
  # return nn.BCEWithLogitsLoss()(outputs, targets)
  return nn.CrossEntropyLoss()(outputs, targets)

optimizer = torch.optim.Adam(params=model.parameters(), lr=LEARNING_RATE)

### Training Loop

In [ ]:
def train_model(
    n_epochs,
    training_loader,
    validation_loader,
    model,
    optimizer,
    checkpoint_path,
    best_model_path
):
  # initialize tracker for minimum validation loss
  valid_loss_min = np.Inf

  for epoch in range(1, n_epochs + 1):
    train_loss = 0
    valid_loss = 0
    model.train()

    print('############# Epoch {}: Training Start   #############'.format(epoch))
    
    # training loop
    # index of batch: 0-31
    for index, batch in enumerate(training_loader):
      input_ids = batch['input_ids'].to(device, dtype=torch.long)
      attention_mask = batch['attention_mask'].to(device, dtype=torch.long)
      token_type_ids = batch['token_type_ids'].to(device, dtype=torch.long)
      targets = batch['targets'].to(device, dtype=torch.float)
      outputs = model(input_ids, attention_mask, token_type_ids)
      optimizer.zero_grad()
      loss = loss_fn(outputs, targets)
      optimizer.zero_grad()
      loss.backward()
      optimizer.step()
      train_loss = train_loss + ((1/(index+1)) * (loss.item()-train_loss))

    print(f'############# Epoch {epoch}: Training End     #############')


    print('############# Epoch {}: Validation Start   #############'.format(epoch))
    
    # validation loop
    model.eval()  # set model into evaluatiom mode
    with torch.no_grad():
      for index, batch in enumerate(validation_loader):
        input_ids = batch['input_ids'].to(device, dtype=torch.long)
        attention_mask = batch['attention_mask'].to(device, dtype=torch.long)
        token_type_ids = batch['token_type_ids'].to(device, dtype=torch.long)
        targets = batch['targets'].to(device, dtype=torch.float)
        outputs = model(input_ids, attention_mask, token_type_ids)
        loss = loss_fn(outputs, targets)
        
        valid_loss = valid_loss + ((1/(index+1)) * (loss.item()-valid_loss))
    
      print('############# Epoch {}: Validation End     #############'.format(epoch))
      # calculate average losses
      #print('before cal avg train loss', train_loss)
      train_loss = train_loss/len(training_loader)
      valid_loss = valid_loss/len(validation_loader)
      # print training/validation statistics 
      print('Epoch: {} \tAvgerage Training Loss: {:.6f} \tAverage Validation Loss: {:.6f}'.format(
            epoch, 
            train_loss,
            valid_loss
            ))

      checkpoint = {
          'epoch': epoch + 1,
          'valid_loss_min': valid_loss,
          'state_dict': model.state_dict(),
          'optimizer': optimizer.state_dict()
      }

      save_ckp(checkpoint, False, checkpoint_path, best_model_path)

      ## save the model if validation loss has decreased
      if valid_loss <= valid_loss_min:
        print('Validation loss decreased ({:.6f} --> {:.6f}).  Saving model ...'.format(valid_loss_min,valid_loss))
        # save checkpoint as best model
        save_ckp(checkpoint, True, checkpoint_path, best_model_path)
        valid_loss_min = valid_loss
  
    print('############# Epoch {}  Done   #############\n'.format(epoch))
  
  return model

In [ ]:
# import pandas as pd
# toy_path = '/content/drive/MyDrive/Dissertation/Models/TER/source.csv'
# df_toy = pd.read_csv(toy_path)
# df_toy

In [ ]:
ckpt_path = "/content/drive/MyDrive/Dissertation/Models/TER_final/curr_ckpt"
best_model_path = "/content/drive/MyDrive/Dissertation/Models/TER_final/best_model.pt"

In [ ]:
# trained_model = train_model(EPOCHS, train_data_loader, valid_data_loader, model, optimizer, '/curr_ckpt', '/best.pt')
trained_model = train_model(EPOCHS, train_data_loader, valid_data_loader, model, optimizer, ckpt_path, best_model_path)

############# Epoch 1: Training Start   #############
############# Epoch 1: Training End     #############
############# Epoch 1: Validation Start   #############
############# Epoch 1: Validation End     #############
Epoch: 1 	Avgerage Training Loss: 0.013440 	Average Validation Loss: 0.034823
Validation loss decreased (inf --> 0.034823).  Saving model ...
############# Epoch 1  Done   #############

############# Epoch 2: Training Start   #############
############# Epoch 2: Training End     #############
############# Epoch 2: Validation Start   #############
############# Epoch 2: Validation End     #############
Epoch: 2 	Avgerage Training Loss: 0.010379 	Average Validation Loss: 0.029662
Validation loss decreased (0.034823 --> 0.029662).  Saving model ...
############# Epoch 2  Done   #############

############# Epoch 3: Training Start   #############
############# Epoch 3: Training End     #############
############# Epoch 3: Validation Start   #############
############# Epo

## Testing

This is just a toy testing section for fast checking.

Detailed testing procedure is introduced in the "TER Testing" colab notebook.

In [ ]:
# testing
example = df_test['UTT'][1]
encodings = tokenizer.encode_plus(
    example,
    None,
    add_special_tokens=True,
    max_length=MAX_LEN,
    padding='max_length',
    return_token_type_ids=True,
    truncation=True,
    return_attention_mask=True,
    return_tensors='pt'
)
model.eval()
with torch.no_grad():
    input_ids = encodings['input_ids'].to(device, dtype=torch.long)
    attention_mask = encodings['attention_mask'].to(device, dtype=torch.long)
    token_type_ids = encodings['token_type_ids'].to(device, dtype=torch.long)
    output = model(input_ids, attention_mask, token_type_ids)
    final_output = torch.sigmoid(output).cpu().detach().numpy().tolist()
    print(final_output)
    print(df_train_true.columns[1:].to_list()[int(np.argmax(final_output, axis=1))])

[[0.8898603916168213, 0.11210853606462479, 0.4700775444507599, 0.5348953604698181]]
neu


In [ ]:
# torch.cuda.empty_cache()

In [ ]:
df_test

,UTT,neu,ang,sad,hap
0,He was only twenty four and just so many thing...,0,0,1,0
1,I know but you just asked me to-,1,0,0,0
2,I'd rather not remember some things. I'd rath...,0,0,1,0
3,and then something like this happens you know?,0,0,1,0
4,"Yeah, I guess I could.",0,0,1,0
...,...,...,...,...,...
1039,Oh really. It's a pity you didn't have any br...,0,1,0,0
1040,I think that,1,0,0,0
1041,"Yeah. I like to know about history, 'cause, y...",0,0,0,1
1042,"You know, we could get a really good debate go...",0,1,0,0


## Reference

* [IEMOCAP dataset](https://sail.usc.edu/iemocap/)

* [Multi-Label Classifition with BERT](https://github.com/theartificialguy/NLP-with-Deep-Learning/tree/master/BERT/Multi%20Label%20Text%20Classification%20using%20BERT%20PyTorch)

* [Multimodal Emotion Recognition with High-Level Speech and Text Features](https://ieeexplore.ieee.org/abstract/document/9688036)